# Analyse de la Répartition des Élus par Sexe et Catégorie Socio-Professionnelle

L'analyse présentée dans ce notebook se base sur le Répertoire National des Élus publié par le gouvernement français, disponible à l'adresse https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv. Ce jeu de données fournit des informations détaillées sur les élus en France, y compris leurs données personnelles, professionnelles et les détails de leurs mandats. L'objectif de cette analyse est de préparer les données pour un diagramme circulaire à deux niveaux représentant la répartition des élus par sexe et catégorie socio-professionnelle, afin de mieux comprendre la composition démographique et socio-professionnelle des élus en France.

L'analyse de la répartition des élus par sexe et catégorie socio-professionnelle est cruciale pour comprendre les dynamiques de représentation au sein des instances politiques françaises. En examinant ces données, nous pouvons identifier les tendances et les disparités dans la représentation des différents groupes socio-professionnels et des sexes. Cela peut éclairer les discussions sur l'égalité des sexes et la diversité dans la sphère politique.


## Méthodologie

La méthodologie employée dans cette analyse consiste à charger le jeu de données dans une base de données DuckDB, puis à exécuter une requête SQL pour agréger les données par sexe et catégorie socio-professionnelle. Les données sont ensuite traitées en utilisant la bibliothèque Pandas pour préparer le diagramme circulaire à deux niveaux. La visualisation est créée en utilisant Plotly Express, avec une attention particulière portée à la sélection des catégories socio-professionnelles les plus représentées pour améliorer la lisibilité du diagramme. La requête SQL utilisée est : 
 SELECT "Code sexe" AS sexe, "Libellé de la catégorie socio-professionnelle" AS csp_libelle, COUNT(*) AS nb_elus FROM loaded_dataset GROUP BY "Code sexe", "Libellé de la catégorie socio-professionnelle" ORDER BY nb_elus DESC.

Les données sont ensuite visualisées sous forme de sunburst chart, permettant de représenter de manière claire et interactive la répartition des élus selon les critères choisis. 

 
## Visualisation des Données 

La visualisation est générée en utilisant le code Python basé sur les bibliothèques Plotly et Pandas.


## Interprétation des Résultats 

L'analyse permet d'identifier les tendances dans la répartition des élus en fonction de leur sexe et de leur catégorie socio-professionnelle, et de mettre en évidence les catégories les plus représentées.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  "Code sexe" AS sexe,
  "Libellé de la catégorie socio-professionnelle" AS csp_libelle,
  COUNT(*) AS nb_elus
FROM 
  loaded_dataset
GROUP BY 
  "Code sexe",
  "Libellé de la catégorie socio-professionnelle"
ORDER BY 
  nb_elus DESC """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est adaptée pour créer des visualisations interactives complexes telles que des diagrammes de sunburst. Ce type de représentation est idéal pour afficher la répartition des données selon plusieurs catégories hiérarchisées, ici le sexe et la catégorie socio-professionnelle des élus. Cela permet une exploration intuitive et détaillée des données.

In [3]:
import pandas as pd
import duckdb as ddb
import plotly.express as px

# Préparation : conserver uniquement les CSP les plus fréquentes si trop catégories
top_csp = df.groupby("csp_libelle")["nb_elus"].sum().nlargest(15).index
df_plot = df[df["csp_libelle"].isin(top_csp)].copy()

dataviz = px.sunburst(
    df_plot,
    path=["sexe", "csp_libelle"],
    values="nb_elus",
    title="Répartition des élus par sexe et CSP - Politique publique élus",
    color="sexe",
    color_discrete_map={"F": "#EF553B", "M": "#636EFA"},
    hover_data={"nb_elus": ":.0f"},
)

dataviz.update_traces(textinfo="label+value+percent parent")
dataviz.update_layout(
    font=dict(family="Roboto, sans-serif", size=12),
    margin=dict(l=20, r=20, t=40, b=20),
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*